# 如何部分格式化提示模板
:::info 前提条件
本指南假定您熟悉以下概念：- [提示模板](/docs/concepts/prompt_templates)
:::
类似于将部分参数绑定到函数上，对[提示模板](/docs/concepts/prompt_templates/)进行"部分化"处理也是有意义的——例如传入所需值的子集，从而创建一个仅需要剩余值子集的新提示模板。
LangChain通过两种方式支持这一点：
1. 使用字符串值进行部分格式化。2. 通过返回字符串值的函数进行部分格式化。
在以下示例中，我们将探讨这两种使用场景的动机，以及如何在LangChain中实现它们。
## 部分字符串处理
想要部分填充提示模板的一个常见用例是，如果你在获取其他变量之前已经获得了提示中的某些变量。例如，假设你有一个提示模板需要两个变量 `foo` 和 `baz`。如果你在链的早期就获得了 `foo` 的值，但 `baz` 的值稍后才得到，那么在整个链中传递这两个变量可能会很不方便。相反，你可以先用 `foo` 的值部分填充提示模板，然后传递这个部分填充的提示模板并直接使用它。以下是一个示例：

In [1]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template("{foo}{bar}")
partial_prompt = prompt.partial(foo="foo")
print(partial_prompt.format(bar="baz"))

foobaz


你也可以仅使用部分变量来初始化提示。

In [2]:
prompt = PromptTemplate(
    template="{foo}{bar}", input_variables=["bar"], partial_variables={"foo": "foo"}
)
print(prompt.format(bar="baz"))

foobaz


## 包含函数的部分
另一种常见用法是使用函数进行部分应用。这种用法的典型场景是当你有一个变量，你希望总是以某种固定方式获取它的值时。日期或时间就是一个很好的例子。假设你有一个提示词，你总是希望其中包含当前日期。你不能在提示词中硬编码这个日期，而每次都与其他输入变量一起传递它又很不方便。在这种情况下，能够用一个总是返回当前日期的函数来部分应用这个提示词就非常方便了。

In [4]:
from datetime import datetime


def _get_datetime():
    now = datetime.now()
    return now.strftime("%m/%d/%Y, %H:%M:%S")


prompt = PromptTemplate(
    template="Tell me a {adjective} joke about the day {date}",
    input_variables=["adjective", "date"],
)
partial_prompt = prompt.partial(date=_get_datetime)
print(partial_prompt.format(adjective="funny"))

Tell me a funny joke about the day 04/21/2024, 19:43:57


你也可以仅使用部分变量初始化提示，这在此工作流程中通常更有意义。

In [5]:
prompt = PromptTemplate(
    template="Tell me a {adjective} joke about the day {date}",
    input_variables=["adjective"],
    partial_variables={"date": _get_datetime},
)
print(prompt.format(adjective="funny"))

Tell me a funny joke about the day 04/21/2024, 19:43:57


## 后续步骤
你现在已经学会了如何将变量部分应用到你的提示模板中。
接下来，请查阅本节中关于提示模板的其他操作指南，例如[如何向提示模板添加少量示例](/docs/how_to/few_shot_examples_chat)。